In [4]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "browser"  # Open in browser for better rendering

# Get all the files
file_paths = [
    "rst_delay_results_qsize_2.txt",
    "rst_delay_results_qsize_5.txt",
    "rst_delay_results_qsize_7.txt",
    "rst_delay_results_qsize_10.txt"
]

# Mapping file names to Mbps values
file_label_mapping = {
    "2": "2Mbps",
    "5": "5Mbps",
    "7": "7Mbps",
    "10": "10Mbps"
}

# Define a marker mapping for each Mbps value
marker_map = {
    "2Mbps": "circle",
    "5Mbps": "diamond",
    "7Mbps": "square",
    "10Mbps": "x"
}

# Create an empty dataframe to store all results
data_list = []

# Read and process each file
for file_path in file_paths:
    df = pd.read_csv(file_path, sep="\t")
    
    # Compute the difference between "h" and "l" for each packetNum
    df_diff = df.pivot(index="qsize", columns="Entropy", values="RST_Delay_Difference(s)")
    df_diff["Difference"] = df_diff["h"] - df_diff["l"]
    
    # Extract file identifier and use mapping
    label = file_path.replace("rst_delay_results_qsize_", "").replace(".txt", "")
    df_diff["File"] = file_label_mapping[label]  # Assign Mbps label
    
    data_list.append(df_diff.reset_index())

# Combine all data
combined_df = pd.concat(data_list, ignore_index=True)

# Plot the data with custom markers
fig = px.line(
    combined_df,
    x="qsize",
    y="Difference",
    color="File",  # Legend uses Mbps values
    markers=True,  # Enable markers
    labels={
        "qsize": r"$\mathrm{Queue Size}$", 
        "Difference": r"$\mathrm{\Delta t_H - \Delta t_L}\,(s)$", 
        "File": "Bandwidth"
    },
    symbol="File"  # Assign different markers based on Mbps value
)

# Apply the custom marker mapping
fig.for_each_trace(lambda trace: trace.update(
    marker_symbol=marker_map[trace.name],  # Assign marker style
    name=r"$\mathrm{" + trace.name.replace("Mbps", r"\ Mbps") + "}$"  # Fix escape sequence
))

# Update layout settings
fig.update_layout(
    width=600,
    height=450,
    plot_bgcolor='white',
    
    legend=dict(
        title="",  # No legend title
        y=0.5,
        xanchor="left",
        x=1.02,
        font=dict(
            family="Arial",
            size=20
        ),
        borderwidth=1.5
    ),
    
    xaxis_title_font=dict(size=24),  # Increase x-axis title font size
    yaxis_title_font=dict(size=24)   # Increase y-axis title font size

)

fig.update_xaxes(gridcolor='lightgrey', gridwidth=1)
fig.update_yaxes(gridcolor='lightgrey', gridwidth=1)

fig.add_shape(
    type="rect",
    xref="paper",
    yref="paper",
    x0=0,
    y0=0,
    x1=1.0,
    y1=1.0,
    line=dict(color="lightgrey", width=1)
)

fig.update_xaxes(range=[0, None])  # Start x-axis from 0
fig.update_yaxes(range=[0, None])  # Start y-axis from 0

# Save as PDF
pio.write_image(fig, 'qSize_4c.pdf')